In [ ]:
# train

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool
from framework import MyFrame
from loss import dice_bce_loss
from data import ImageFolder

if __name__ == '__main__':
    SHAPE = (1024,1024)
    ROOT = 'dataset/train/'
    imagelist = filter(lambda x: x.find('sat')!=-1, os.listdir(ROOT))
    trainlist = list(map(lambda x: x[:-8], imagelist))
    NAME = 'log01_dink34'
    BATCHSIZE_PER_CARD = 4

    solver = MyFrame(DinkNet34, dice_bce_loss, 2e-4)
    batchsize = torch.cuda.device_count() * BATCHSIZE_PER_CARD

    dataset = ImageFolder(trainlist, ROOT)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batchsize,
        shuffle=True,
        num_workers=4)

    mylog = open('logs/'+NAME+'.log','w',encoding='utf-8')
    tic = time()
    no_optim = 0
    total_epoch = 300
    train_epoch_best_loss = 100.

    for epoch in range(1, total_epoch + 1):
        data_loader_iter = iter(data_loader)
        train_epoch_loss = 0
        for img, mask in data_loader_iter:
            solver.set_input(img, mask)
            train_loss = solver.optimize()
            train_epoch_loss += train_loss
        train_epoch_loss /= len(data_loader_iter)
        print (mylog, '********')
        print (mylog, 'epoch:',epoch,'    time:',int(time()-tic))
        print (mylog, 'train_loss:',train_epoch_loss)
        print (mylog, 'SHAPE:',SHAPE)
        mylog.write('********'+'\n')
        mylog.write('epoch:'+str(epoch)+'    time:'+str(int(time()-tic))+'\n')
        mylog.write('train_loss:'+str(train_epoch_loss)+'\n')
        mylog.write('SHAPE:'+str(SHAPE)+'\n')
        print ('********')
        print ('epoch:',epoch,'    time:',int(time()-tic))
        print ('train_loss:',train_epoch_loss)
        print ('SHAPE:',SHAPE)
        
        if train_epoch_loss >= train_epoch_best_loss:
            no_optim += 1
        else:
            no_optim = 0
            train_epoch_best_loss = train_epoch_loss
            solver.save('weights/'+NAME+'.th')
        if no_optim > 6:
            print (mylog, 'early stop at %d epoch' % epoch)
            mylog.write('early stop at '+str(epoch)+' epoch'+'\n')
            print ('early stop at %d epoch' % epoch)
            break
        if no_optim > 3:
            if solver.old_lr < 5e-7:
                break
            solver.load('weights/'+NAME+'.th')
            solver.update_lr(5.0, factor = True, mylog = mylog)
        mylog.flush()

    print (mylog, 'Finish!')
    mylog.write('Finish!')
    print ('Finish!')
    mylog.close()

In [2]:
# test

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool

BATCHSIZE_PER_CARD = 4

class TTAFrame():
    def __init__(self, net):
        self.net = net().cuda()
        self.net = torch.nn.DataParallel(self.net, device_ids=range(torch.cuda.device_count()))
        
    def test_one_img_from_path(self, path, evalmode = True):
        if evalmode:
            self.net.eval()
        batchsize = torch.cuda.device_count() * BATCHSIZE_PER_CARD
        if batchsize >= 8:
            return self.test_one_img_from_path_1(path)
        elif batchsize >= 4:
            return self.test_one_img_from_path_2(path)
        elif batchsize >= 2:
            return self.test_one_img_from_path_4(path)

    def test_one_img_from_path_8(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]
        
        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)
        
        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())
        
        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]
        
        return mask2

    def test_one_img_from_path_4(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]
        
        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)
        
        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())
        
        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]
        
        return mask2
    
    def test_one_img_from_path_2(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = img3.transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        img6 = img4.transpose(0,3,1,2)
        img6 = np.array(img6, np.float32)/255.0 * 3.2 -1.6
        img6 = V(torch.Tensor(img6).cuda())
        
        maska = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        maskb = self.net.forward(img6).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3
    
    def test_one_img_from_path_1(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = np.concatenate([img3,img4]).transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        
        mask = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        mask1 = mask[:4] + mask[4:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3

    def load(self, path):
        self.net.load_state_dict(torch.load(path))
        
source = 'dataset/test/'
#source = 'dataset/valid/'
val = os.listdir(source)
solver = TTAFrame(DinkNet34)
solver.load('weights/log01_dink34.th')
tic = time()
target = 'submits/log01_dink34/'
os.mkdir(target)
for i,name in enumerate(val):
    if i%10 == 0:
        print (i/10, '    ','%.2f'%(time()-tic))
    mask = solver.test_one_img_from_path(source+name)
    mask[mask>4.0] = 255
    mask[mask<=4.0] = 0
    mask = np.concatenate([mask[:,:,None],mask[:,:,None],mask[:,:,None]],axis=2)
    cv2.imwrite(target+name[:-7]+'mask.png',mask.astype(np.uint8))


0.0      0.04
1.0      8.79
2.0      15.90
3.0      22.89
4.0      30.36
5.0      37.88
6.0      45.47
7.0      53.16
8.0      60.21
9.0      67.67
10.0      74.31
11.0      81.23
12.0      88.51
13.0      96.16
14.0      103.56
15.0      111.13
16.0      118.61
17.0      126.09
18.0      133.34
19.0      139.97
20.0      146.12
21.0      152.72
22.0      159.74
23.0      167.05
24.0      174.41
25.0      181.75
26.0      188.88
27.0      195.61
28.0      202.62
29.0      209.14
30.0      215.71
31.0      222.47
32.0      229.13
33.0      235.89
34.0      242.14
35.0      248.78
36.0      255.13
37.0      262.07
38.0      268.35
39.0      274.82
40.0      281.41
41.0      287.74
42.0      294.21
43.0      300.88
44.0      307.31
45.0      313.88
46.0      320.55
47.0      327.04
48.0      333.46
49.0      339.90
50.0      346.03
51.0      352.27
52.0      358.83
53.0      365.31
54.0      371.73
55.0      377.88
56.0      384.28
57.0      390.65
58.0      396.91
59.0      403.44
60.0  

In [4]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool

BATCHSIZE_PER_CARD = 4

class TTAFrame():
    def __init__(self, net):
        self.net = net().cuda()
        self.net = torch.nn.DataParallel(self.net, device_ids=range(torch.cuda.device_count()))
        
    def test_one_img_from_path(self, path, evalmode = True):
        if evalmode:
            self.net.eval()
        batchsize = torch.cuda.device_count() * BATCHSIZE_PER_CARD
        if batchsize >= 8:
            return self.test_one_img_from_path_1(path)
        elif batchsize >= 4:
            return self.test_one_img_from_path_2(path)
        elif batchsize >= 2:
            return self.test_one_img_from_path_4(path)

    def test_one_img_from_path_8(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]
        
        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)
        
        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())
        print(img1.shape)
        print(img2.shape)
        print(img3.shape)
        print(img4.shape)
        
        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]
        
        return mask2

    def test_one_img_from_path_4(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]
        
        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)
        
        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())
        print(img1.shape)
        print(img2.shape)
        print(img3.shape)
        print(img4.shape)
        
        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]
        
        return mask2
    
    def test_one_img_from_path_2(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = img3.transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        img6 = img4.transpose(0,3,1,2)
        img6 = np.array(img6, np.float32)/255.0 * 3.2 -1.6
        img6 = V(torch.Tensor(img6).cuda())
        print(img5.shape)
        print(img6.shape)
        
        maska = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        maskb = self.net.forward(img6).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3
    
    def test_one_img_from_path_1(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = np.concatenate([img3,img4]).transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        print(img5.shape)
        
        mask = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        mask1 = mask[:4] + mask[4:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3

    def load(self, path):
        self.net.load_state_dict(torch.load(path))
        
#source = 'dataset/test/'
#source = 'dataset/valid/'
source = 'dataset/forONNX/'
val = os.listdir(source)
solver = TTAFrame(DinkNet34)
solver.load('weights/log01_dink34.th')
tic = time()
#target = 'submits/log01_dink34/'
target = 'submits/forONNX/'
os.mkdir(target)
print('begin!!!')
for i,name in enumerate(val):
    if i%10 == 0:
        print (i/10, '    ','%.2f'%(time()-tic))
    mask = solver.test_one_img_from_path(source+name)
    mask[mask>4.0] = 255
    mask[mask<=4.0] = 0
    mask = np.concatenate([mask[:,:,None],mask[:,:,None],mask[:,:,None]],axis=2)
    cv2.imwrite(target+name[:-7]+'mask.png',mask.astype(np.uint8))

#x = 

begin!!!
0.0      0.01
torch.Size([4, 3, 1024, 1024])
torch.Size([4, 3, 1024, 1024])


In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool

BATCHSIZE_PER_CARD = 4

class TTAFrame():
    def __init__(self, net):
        self.net = net().cuda()
        self.net = torch.nn.DataParallel(self.net, device_ids=range(torch.cuda.device_count()))
        
    def test_one_img_from_path(self, path, evalmode = True):
        if evalmode:
            self.net.eval()
        batchsize = torch.cuda.device_count() * BATCHSIZE_PER_CARD
        if batchsize >= 8:
            return self.test_one_img_from_path_1(path)
        elif batchsize >= 4:
            return self.test_one_img_from_path_2(path)
        elif batchsize >= 2:
            return self.test_one_img_from_path_4(path)

    def test_one_img_from_path_8(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]
        
        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)
        
        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())
        
        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]
        
        return mask2

    def test_one_img_from_path_4(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.array(img1)[:,:,::-1]
        img4 = np.array(img2)[:,:,::-1]
        
        img1 = img1.transpose(0,3,1,2)
        img2 = img2.transpose(0,3,1,2)
        img3 = img3.transpose(0,3,1,2)
        img4 = img4.transpose(0,3,1,2)
        
        img1 = V(torch.Tensor(np.array(img1, np.float32)/255.0 * 3.2 -1.6).cuda())
        img2 = V(torch.Tensor(np.array(img2, np.float32)/255.0 * 3.2 -1.6).cuda())
        img3 = V(torch.Tensor(np.array(img3, np.float32)/255.0 * 3.2 -1.6).cuda())
        img4 = V(torch.Tensor(np.array(img4, np.float32)/255.0 * 3.2 -1.6).cuda())
        
        maska = self.net.forward(img1).squeeze().cpu().data.numpy()
        maskb = self.net.forward(img2).squeeze().cpu().data.numpy()
        maskc = self.net.forward(img3).squeeze().cpu().data.numpy()
        maskd = self.net.forward(img4).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,::-1] + maskc[:,:,::-1] + maskd[:,::-1,::-1]
        mask2 = mask1[0] + np.rot90(mask1[1])[::-1,::-1]
        
        return mask2
    
    def test_one_img_from_path_2(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = img3.transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        img6 = img4.transpose(0,3,1,2)
        img6 = np.array(img6, np.float32)/255.0 * 3.2 -1.6
        img6 = V(torch.Tensor(img6).cuda())
        
        maska = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        maskb = self.net.forward(img6).squeeze().cpu().data.numpy()
        
        mask1 = maska + maskb[:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3
    
    def test_one_img_from_path_1(self, path):
        img = cv2.imread(path)#.transpose(2,0,1)[None]
        
        img90 = np.array(np.rot90(img))
        img1 = np.concatenate([img[None],img90[None]])
        img2 = np.array(img1)[:,::-1]
        img3 = np.concatenate([img1,img2])
        img4 = np.array(img3)[:,:,::-1]
        img5 = np.concatenate([img3,img4]).transpose(0,3,1,2)
        img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
        img5 = V(torch.Tensor(img5).cuda())
        
        mask = self.net.forward(img5).squeeze().cpu().data.numpy()#.squeeze(1)
        mask1 = mask[:4] + mask[4:,:,::-1]
        mask2 = mask1[:2] + mask1[2:,::-1]
        mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
        
        return mask3

    def load(self, path):
        self.net.load_state_dict(torch.load(path))
        
#source = 'dataset/test/'
#source = 'dataset/valid/'
source = 'dataset/forONNX/'
val = os.listdir(source)
solver = TTAFrame(DinkNet34)
solver.load('weights/log01_dink34.th')
tic = time()
#target = 'submits/log01_dink34/'
target = 'submits/forONNX/'
os.mkdir(target)
for i,name in enumerate(val):
    if i%10 == 0:
        print (i/10, '    ','%.2f'%(time()-tic))
    mask = solver.test_one_img_from_path(source+name)
    mask[mask>4.0] = 255
    mask[mask<=4.0] = 0
    mask = np.concatenate([mask[:,:,None],mask[:,:,None],mask[:,:,None]],axis=2)
    cv2.imwrite(target+name[:-7]+'mask.png',mask.astype(np.uint8))

# x = torch.randn(4,3,1024,1024,device='cuda')

# print('begin ONNX')

# with torch.no_grad():
#     torch.onnx.export(
#         solver.net.module,
#         x,
#         "dlinknet.onnx",
#         opset_version=12,
#         input_names=['input'],
#         output_names=['output'])
    
# print('output ONNX')
        

0.0      0.00
[[[7.7557093e-03 4.9406732e-03 4.1647963e-03 ... 1.8974004e-04
   2.8535060e-04 1.0521074e-03]
  [2.2293811e-03 2.0356083e-03 2.5193407e-03 ... 4.9274729e-04
   9.4200863e-04 1.0172684e-03]
  [1.6473376e-03 2.1299426e-03 2.0775364e-03 ... 5.3662079e-04
   7.7645166e-04 7.2671566e-04]
  ...
  [1.3130100e-05 8.2566550e-07 6.1624719e-06 ... 3.6434369e-04
   3.3950919e-04 2.4978683e-04]
  [1.4486019e-05 3.4350580e-06 1.1493468e-05 ... 5.0009653e-04
   5.8114511e-04 2.4562297e-04]
  [3.3174943e-05 3.8948565e-05 3.6077436e-05 ... 3.4465120e-04
   3.3421963e-04 4.1164519e-04]]

 [[8.0588034e-05 1.2698638e-05 8.9890145e-06 ... 1.3959416e-03
   1.6096047e-03 3.0852626e-03]
  [2.5199164e-05 7.5538283e-06 1.4483147e-05 ... 1.6197984e-03
   1.5389161e-03 1.5818164e-03]
  [1.9684674e-05 2.1697026e-05 3.2815860e-05 ... 1.4140366e-03
   1.1378527e-03 1.1329742e-03]
  ...
  [8.5350570e-05 6.8428213e-05 1.9281768e-04 ... 2.6991448e-04
   2.5775243e-04 1.8459788e-04]
  [5.9945236e-05 1.048

In [2]:
import onnx

onnx_model = onnx.load("dlinknet.onnx")
try:
    onnx.checker.check_model(onnx_model)
except Exception:
    print("Model incorrect")
else:
    print("Model correct")

Model correct


In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import onnxruntime

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool
        
#source = 'dataset/test/'
#source = 'dataset/valid/'
source = 'dataset/forONNX/'
val = os.listdir(source)
ort_session = onnxruntime.InferenceSession("dlinknet.onnx", providers=onnxruntime.get_available_providers())
tic = time()
#target = 'submits/log01_dink34/'
target = 'submits/forONNX/'
os.mkdir(target)
for i,name in enumerate(val):
    if i%10 == 0:
        print (i/10, '    ','%.2f'%(time()-tic))
    path = source+name
    
    #opencv读取.jpg
    img = cv2.imread(path)#.transpose(2,0,1)[None]
    print('img:')
    print(img.shape)
    #将数组旋转90度
    img90 = np.array(np.rot90(img))
    print('img90:')
    print(img90.shape)
    #拼接两个数组, numpy.array[None] == npmpy.array.newaxis
    img1 = np.concatenate([img[None],img90[None]])
    print('img1:')
    print(img1.shape)
    #数组冒号取值
    img2 = np.array(img1)[:,::-1]
    print('img2:')
    print(img2.shape)
    #拼接
    img3 = np.concatenate([img1,img2])
    print('img3:')
    print(img3.shape)
    #数组冒号取值
    img4 = np.array(img3)[:,:,::-1]
    print('img4:')
    print(img4.shape)
    #四个维度转置
    img5 = img3.transpose(0,3,1,2)
    print('img5:')
    print(img5.shape)
    
    img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
    print('_img5:')
    print(img5)
    print(img5.shape)
    img5 = V(torch.Tensor(img5).cuda())
    
    #四个维度转置
    img6 = img4.transpose(0,3,1,2)
    print('img6:')
    print(img6.shape)
    
    img6 = np.array(img6, np.float32)/255.0 * 3.2 -1.6
    print('_img6:')
    print(img6)
    print(img6.shape)
    img6 = V(torch.Tensor(img6).cuda())

    print('Begin')
    ort_inputs = {'input': torch.Tensor(img5).cpu().numpy()}
    ort_output = ort_session.run(['output'],ort_inputs)[0]
    maska = ort_output.squeeze()
    
    print('Next')
    ort_inputs = {'input': torch.Tensor(img6).cpu().numpy()}
    ort_output = ort_session.run(['output'],ort_inputs)[0]
    maskb = ort_output.squeeze()

    print('Drawing')
    mask1 = maska + maskb[:,:,::-1]
    mask2 = mask1[:2] + mask1[2:,::-1]
    mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
    mask = mask3
    mask[mask>4.0] = 255
    mask[mask<=4.0] = 0
    mask = np.concatenate([mask[:,:,None],mask[:,:,None],mask[:,:,None]],axis=2)
    cv2.imwrite(target+name[:-7]+'mask.png',mask.astype(np.uint8))
    
print('Finished')

0.0      0.04
img:
(1024, 1024, 3)
img90:
(1024, 1024, 3)
img1:
(2, 1024, 1024, 3)
img2:
(2, 1024, 1024, 3)
img3:
(4, 1024, 1024, 3)
img4:
(4, 1024, 1024, 3)
img5:
(4, 3, 1024, 1024)
_img5:
[[[[ 0.21960795  0.21960795  0.269804   ...  0.8219608   1.0603923
     1.0854903 ]
   [ 0.1694119   0.2070589   0.269804   ...  1.0227453   1.0227453
     0.9349021 ]
   [ 0.13176477  0.15686285  0.19450986 ...  0.985098    1.0227453
     0.96000016]
   ...
   [-0.03137255 -0.03137255 -0.00627446 ... -0.9098039  -0.7717647
    -0.8596079 ]
   [ 0.13176477  0.08156872  0.05647063 ... -0.8847059  -0.8847059
    -0.8847059 ]
   [ 0.21960795  0.1694119   0.08156872 ... -0.75921565 -0.80941176
    -0.8345098 ]]

  [[ 0.6964706   0.6964706   0.7466668  ...  1.0980393   1.3113726
     1.3364707 ]
   [ 0.6462747   0.6839217   0.7466668  ...  1.2988235   1.2611765
     1.1858824 ]
   [ 0.60862744  0.6337255   0.67137253 ...  1.2235295   1.2611765
     1.1984314 ]
   ...
   [ 0.4580394   0.4580394   0.483137

AttributeError: 'NoneType' object has no attribute 'shape'

In [3]:
import onnxruntime

print(onnxruntime.get_device())

print(onnxruntime.get_available_providers())

ort_session = onnxruntime.InferenceSession("dlinknet.onnx",providers=onnxruntime.get_available_providers())
print(ort_session.get_providers())

GPU
['CPUExecutionProvider', 'CUDAExecutionProvider']
['CPUExecutionProvider', 'CUDAExecutionProvider']


In [6]:
import torch
print(torch.__version__)

print(torch.version.cuda)
print(torch.backends.cudnn.version())

1.12.0+cu102
10.2
7605


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool
        
#source = 'dataset/test/'
#source = 'dataset/valid/'
source = 'dataset/forONNX/'
val = os.listdir(source)
tic = time()
#target = 'submits/log01_dink34/'
target = 'submits/forONNX/'
for i,name in enumerate(val):
    if i%10 == 0:
        print (i/10, '    ','%.2f'%(time()-tic))
    path = source+name
    
    #opencv读取.jpg
    img = cv2.imread(path)#.transpose(2,0,1)[None]
    print('img:')
    print(img.shape)
    #将数组旋转90度
    img90 = np.array(np.rot90(img))
    print('img90:')
    print(img90.shape)
    #拼接两个数组, numpy.array[None] == npmpy.array.newaxis
    img1 = np.concatenate([img[None],img90[None]])
    print('img1:')
    print(img1.shape)
    #数组冒号取值
    img2 = np.array(img1)[:,::-1]
    print('img2:')
    print(img2.shape)
    print(img2)
    #拼接
    img3 = np.concatenate([img1,img2])
    print('img3:')
    print(img3.shape)
    #数组冒号取值
    img4 = np.array(img3)[:,:,::-1]
    print('img4:')
    print(img4.shape)
    #四个维度转置
    img5 = img3.transpose(0,3,1,2)
    print('img5:')
    print(img5.shape)
    
    img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
    print('_img5:')
    print(img5)
    print(img5.shape)
    img5 = V(torch.Tensor(img5).cuda())
    
    #四个维度转置
    img6 = img4.transpose(0,3,1,2)
    print('img6:')
    print(img6.shape)
    
    img6 = np.array(img6, np.float32)/255.0 * 3.2 -1.6
    print('_img6:')
    print(img6)
    print(img6.shape)
    img6 = V(torch.Tensor(img6).cuda())
    
print('Finished')

In [11]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable as V

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import onnxruntime

from time import time

from networks.unet import Unet
from networks.dunet import Dunet
from networks.dinknet import LinkNet34, DinkNet34, DinkNet50, DinkNet101, DinkNet34_less_pool
        
#source = 'dataset/test/'
#source = 'dataset/valid/'
source = 'dataset/forONNX/'
val = os.listdir(source)
ort_session = onnxruntime.InferenceSession("dlinknet.onnx", providers=onnxruntime.get_available_providers())
tic = time()
#target = 'submits/log01_dink34/'
target = 'submits/forONNX/'
for i,name in enumerate(val):
    if i%10 == 0:
        print (i/10, '    ','%.2f'%(time()-tic))
    path = source+name
    
    #opencv读取.jpg
    img = cv2.imread(path)#.transpose(2,0,1)[None]
    #将数组旋转90度
    img90 = np.array(np.rot90(img))
    #拼接两个数组, numpy.array[None] == npmpy.array.newaxis
    img1 = np.concatenate([img[None],img90[None]])
    #数组冒号取值
    img2 = np.array(img1)[:,::-1]
    #拼接
    img3 = np.concatenate([img1,img2])
    #数组冒号取值
    img4 = np.array(img3)[:,:,::-1]
    #四个维度转置
    img5 = img3.transpose(0,3,1,2)
    
    img5 = np.array(img5, np.float32)/255.0 * 3.2 -1.6
    img5 = V(torch.Tensor(img5).cuda())
    
    #四个维度转置
    img6 = img4.transpose(0,3,1,2)
    
    img6 = np.array(img6, np.float32)/255.0 * 3.2 -1.6
    img6 = V(torch.Tensor(img6).cuda())

    print('Begin')
    ort_inputs = {'input': torch.Tensor(img5).cpu().numpy()}
    ort_output = ort_session.run(['output'],ort_inputs)[0]
    maska = ort_output.squeeze()
    
    print('Next')
    ort_inputs = {'input': torch.Tensor(img6).cpu().numpy()}
    ort_output = ort_session.run(['output'],ort_inputs)[0]
    maskb = ort_output.squeeze()

    print('Drawing')
    mask1 = maska + maskb[:,:,::-1]
    mask2 = mask1[:2] + mask1[2:,::-1]
    mask3 = mask2[0] + np.rot90(mask2[1])[::-1,::-1]
    print('np.rot90(mask2[1])')
    print(np.rot90(mask2[1]).shape)
    print(np.rot90(mask2[1]))
    print('np.rot90(mask2[1])[::-1,::-1]')
    print(np.rot90(mask2[1])[::-1,::-1].shape)
    print(np.rot90(mask2[1])[::-1,::-1])
    print('mask3')
    print(mask3.shape)
    print(mask3)
    mask = mask3
    mask[mask>4.0] = 255
    mask[mask<=4.0] = 0
    print('mask[:,:,None]')
    print(mask[:,:,None].shape)
    print(mask[:,:,None])
    mask = np.concatenate([mask[:,:,None],mask[:,:,None],mask[:,:,None]],axis=2)
    print('mask')
    print(mask.shape)
    print(mask)
    print('mask.astype(np.uint8)')
    print(mask.astype(np.uint8).shape)
    print(mask.astype(np.uint8))
    
print('Finished')

0.0      0.00
Begin
Next
Drawing
np.rot90(mask2[1])
(1024, 1024)
[[2.3919344e-04 1.7073750e-04 1.3250113e-04 ... 1.1347830e-03
  1.3451576e-03 2.7144849e-03]
 [1.0436773e-04 5.7905912e-05 5.3852797e-05 ... 1.5551150e-03
  2.1417737e-03 1.8460453e-03]
 [8.1956387e-05 5.1110983e-05 6.2614679e-05 ... 1.6806722e-03
  1.9291043e-03 1.9601285e-03]
 ...
 [1.4883459e-02 1.4063716e-02 1.3613313e-02 ... 1.1982918e-03
  1.2487471e-03 1.5054047e-03]
 [1.4721632e-02 1.0966986e-02 9.4214678e-03 ... 1.2077689e-03
  1.3832748e-03 1.6854703e-03]
 [1.9852996e-02 7.0895553e-03 6.4090788e-03 ... 1.0522902e-03
  1.1832118e-03 2.9730797e-03]]
np.rot90(mask2[1])[::-1,::-1]
(1024, 1024)
[[2.9730797e-03 1.1832118e-03 1.0522902e-03 ... 6.4090788e-03
  7.0895553e-03 1.9852996e-02]
 [1.6854703e-03 1.3832748e-03 1.2077689e-03 ... 9.4214678e-03
  1.0966986e-02 1.4721632e-02]
 [1.5054047e-03 1.2487471e-03 1.1982918e-03 ... 1.3613313e-02
  1.4063716e-02 1.4883459e-02]
 ...
 [1.9601285e-03 1.9291043e-03 1.6806722e-03 

ValueError: Axes=(0, 1) out of range for array of ndim=0.